## Pyculib FFT example

In [1]:
from copy import deepcopy

In [2]:
import pyculib.fft
import numba.cuda
import numpy as np

@numba.cuda.jit
def apply_mask(frame, mask):
    i, j = numba.cuda.grid(2)
    frame[i, j] *= mask[i, j]

In [3]:
tpb = (16, 16)
blocks = (720//16, 1280//16)

# make some random test data
frame = np.random.uniform(0, 255.0, 720*1280).astype(np.float32).reshape((720, 1280))
mask = np.random.uniform(0, 255.0, 720*1280).astype(np.float32).reshape((720, 1280))

out = np.empty_like(mask, dtype=np.complex64)
gpu_temp = numba.cuda.to_device(out) # make GPU array
gpu_mask = numba.cuda.to_device(mask) # make GPU array

pyculib.fft.fft(frame.astype(np.complex64), gpu_temp) # implied host->device
apply_mask[blocks, tpb](gpu_temp, gpu_mask) # all on device
pyculib.fft.ifft(gpu_temp, out) # implied device->host

# Note that out needs to be normalized before display

array([[  2.69412393e+10 -3.19479398e+09j,
          1.47760005e+10 -5.18573517e+09j,
          9.48282573e+09 -2.18189158e+09j, ...,
          1.71281992e+10 -1.31452800e+09j,
          1.82950523e+10 -4.43124352e+08j,
          2.81224008e+10 -2.51320397e+09j],
       [  1.16347044e+10 -6.14364570e+09j,
          2.99841782e+10 +5.01267328e+08j,
          1.25970739e+10 +4.74879078e+09j, ...,
          8.56365517e+09 +2.27370445e+09j,
          3.10401208e+10 +1.94992077e+09j,
          2.98579292e+10 -2.91163930e+09j],
       [  1.95428086e+10 +3.77971712e+08j,
          2.11252736e+10 +7.05626573e+09j,
          1.42609551e+10 -6.65683558e+09j, ...,
          1.01301862e+10 -1.83273651e+09j,
          1.92173220e+10 +4.90336307e+09j,
          2.92726723e+10 -1.59420877e+09j],
       ..., 
       [  1.54578227e+10 -3.53495501e+09j,
          2.71120159e+10 -2.98723584e+08j,
          1.51615498e+10 -2.73297664e+08j, ...,
          4.28769362e+10 -4.42258995e+09j,
          3.554074

## Use the same function on CPU and GPU

Here we can use the `clamp` function on the GPU as a device function without using the `cuda.jit` decorator:

In [4]:
@numba.jit
def clamp(x, xmin, xmax):
    if x < xmin:
        return xmin
    elif x > xmax:
        return xmax
    else:
        return x
    
@numba.cuda.jit
def clamp_array(x, xmin, xmax, out):
    # Assuming 1D array
    start = numba.cuda.grid(1)
    stride = numba.cuda.gridsize(1)
    
    for i in range(start, x.shape[0], stride):
        out[i] = clamp(x[i], xmin, xmax)  # call "CPU" function here

In [5]:
def clamp_array2(x, xmin, xmax, out):
  
    for i in range(0, x.shape[0], 1):
        out[i] = clamp(x[i], xmin, xmax)  # call "CPU" function here

In [24]:
x = np.linspace(-10, 10, 100000)
out_device = numba.cuda.device_array_like(x)
out_device2 = deepcopy(x)
%timeit clamp_array[64, 64](x, -1, 1, out_device)
# %timeit clamp_array2(x, -1, 1, out_device2)
out = out_device.copy_to_host()
print(out[:10])
print(out[500:510])
print(out[-10:])

1000 loops, best of 3: 1.08 ms per loop
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [37]:
import math
import numpy as np

In [46]:
@numba.cuda.jit
def steerGPU(newVertices):
    for n in range(10):
        for i in range(10):
            dx = 2.3 * math.cos(newVertices[n, i - 1, 2])
            dy = 2.3 * math.sin(newVertices[n, i - 1, 2])
            dtheta = 0.25 / 4.0 * np.random.rand(0.0, math.sqrt(0.1))
            newVertices[n, i, 0:2] = newVertices[n, i - 1, 0:2] + 0.1 * np.array([dx, dy])
            newVertices[n, i, 2] = newVertices[n, i - 1, 2] + dtheta
            newVertices[n, i, 3] = i * 0.1
            newVertices[n, i, 4] = dtheta * 4.0
            newVertices[n, i, 5] = i - 1

In [48]:
newVertices = np.zeros((10, 10, 6))
out_device = numba.cuda.device_array_like(newVertices)
%timeit steerGPU[64, 64](out_device)
# out = out_device.copy_to_host()
# print out.shape

TypingError: Caused By:
Traceback (most recent call last):
  File "/home/ankit/anaconda2/envs/pathplanning/lib/python2.7/site-packages/numba/compiler.py", line 235, in run
    stage()
  File "/home/ankit/anaconda2/envs/pathplanning/lib/python2.7/site-packages/numba/compiler.py", line 449, in stage_nopython_frontend
    self.locals)
  File "/home/ankit/anaconda2/envs/pathplanning/lib/python2.7/site-packages/numba/compiler.py", line 805, in type_inference_stage
    infer.propagate()
  File "/home/ankit/anaconda2/envs/pathplanning/lib/python2.7/site-packages/numba/typeinfer.py", line 767, in propagate
    raise errors[0]
TypingError: Invalid usage of * with parameters (float64, array(float64, 2d, C))
Known signatures:
 * (int64, int64) -> int64
 * (int64, uint64) -> int64
 * (uint64, int64) -> int64
 * (uint64, uint64) -> uint64
 * (float32, float32) -> float32
 * (float64, float64) -> float64
 * (complex64, complex64) -> complex64
 * (complex128, complex128) -> complex128
 * parameterize

In [13]:
from RRT import RRT
from Vertex import Vertex

In [14]:
vInit = Vertex(-9.,0.,0.,0.,0.,0)
vGoal = Vertex(9.,0.,0.,10.,0.,0)

alphas = [0.25,0.5,1.0]
obstacleTypes = ['single', 'double']
runTypes = ['rrt','pirrt']
runType = runTypes[1]
useRRTStar = False

In [15]:
rrt = RRT(vInit,vGoal,alpha=alphas[0],obstacleType=obstacleTypes[0])

rrt initialized with (-9.0, 0.0, 0.0, 0.0, 0.0, 0)


In [16]:
@numba.cuda.jit
def extractPathGPU(RRT):
    # Assuming 1D array
    RRT.extractPath()

In [18]:
extractPathGPU[64, 64](rrt)

ValueError: cannot determine Numba type of <class 'RRT.RRT'>

## Making ufuncs with @vectorize

This example shows how to turn a scalar function into a CUDA-accelerated ufunc.

In [ ]:
import numba
import math
import numpy as np

SQRT_TWOPI = np.float32(math.sqrt(2 * math.pi))

@numba.vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian(x, x0, sigma):
    return math.exp(-((x - x0) / sigma)**2 / 2) / SQRT_TWOPI / sigma

In [ ]:
x = np.linspace(-3, 3, 10000, dtype=np.float32)
g = gaussian(x, 0, 1)  # 1D result

x2d = x.reshape((100,100))
g2d = gaussian(x2d, 0, 1) # 2D result

## Dask Example

This is a short example of executing a CUDA kernel with Dask.  In this example, we start a local Dask cluster, but it could just as easily be a remote cluster.

In [ ]:
@numba.cuda.jit
def gpu_cos(x, out):
    # Assuming 1D array
    start = numba.cuda.grid(1)
    stride = numba.cuda.gridsize(1)
    
    for i in range(start, x.shape[0], stride):
        out[i] = math.cos(x[i])
        
def do_cos(x):
    out = numba.cuda.device_array_like(x)
    gpu_cos[64, 64](x, out)
    return out.copy_to_host()

In [ ]:
# check if works locally first
test_x = np.random.uniform(-10, 10, 1000).astype(np.float32)
result = do_cos(test_x)
np.testing.assert_allclose(result, np.cos(test_x), rtol=1e-6)

In [ ]:
# now try remote
from dask.distributed import Client
client = Client() # starts a local cluster

future = client.submit(do_cos, test_x)
gpu_result = future.result()

In [ ]:
np.testing.assert_allclose(gpu_result, np.cos(test_x), rtol=1e-6)